In [12]:
from onnxruntime.training import artifacts
import torch
import onnx
import transformers
# from fastT5 import export_and_get_onnx_model
import onnxruntime.training.api as ort_api

In [2]:
# model = export_and_get_onnx_model("MBZUAI/LaMini-T5-61M")

In [3]:
# pipeline = transformers.pipeline(
#     "text-generation",
#     model="distilbert/distilgpt2",
# )

# transformers_model = transformers.AutoModelForCausalLM.from_pretrained("MBZUAI/LaMini-Cerebras-111M")
# transformers_model = transformers.AutoModel.from_pretrained("facebook/blenderbot-400M-distill")
transformers_model = transformers.AutoModel.from_pretrained("distilbert/distilgpt2")
# transformers_model_gpt = transformers.GPT2Model.from_pretrained("distilgpt2")

In [4]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained("distilgpt2")
# tokenizer = transformers.GPT2Tokenizer.from_pretrained("distilbert/distilgpt2")
inputs = tokenizer("The capital of France is Paris.", return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 464, 3139,  286, 4881,  318, 6342,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [5]:
class FlatModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *local_inputs):
        return self.model(inputs.input_ids, inputs.attention_mask)

# model = FlatModel(pipeline.model)
model = FlatModel(transformers_model)

In [9]:
input_names = ["input_ids", "attention_mask"]
output_names = ["loss", "logits"]

torch.onnx.export(model,
                  (inputs["input_ids"], inputs["attention_mask"]),
                  "distilgpt2.onnx",
                  input_names = input_names, 
                  output_names = output_names,
                  export_params=True,
                  training=torch.onnx.TrainingMode.TRAINING,
                  do_constant_folding=False,
                  opset_version=15,
                  dynamic_axes={
                    "input_ids": {0: "batch_size", 1: "sequence_length"},
                    "attention_mask": {0: "batch_size", 1: "sequence_length"},
                    "logits": {0: "batch_size", 1: "sequence_length"}
                  }
                  )

input_ids: tensor([[ 464, 3139,  286, 4881,  318, 6342,   13]])
past key values: None
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1]])


c:\Users\carolinezhu\AppData\Local\anaconda3\envs\generate-artifacts-new\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:804: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:


In [11]:
requires_grad = []
frozen_params = []
num_named_params = 0
for name, param in model.named_parameters():
    num_named_params += 1
    if param.requires_grad:
        requires_grad.append(name)
    else:
        frozen_params.append(name)

for name, param in model.named_buffers():
    frozen_params.append(name)

# onnx_model = onnx.load("tinyllama.onnx")
onnx_model = onnx.load("distilgpt2.onnx")

artifacts.generate_artifacts(
    onnx_model,
    requires_grad=requires_grad,
    frozen_params=frozen_params,
    optimizer=artifacts.OptimType.AdamW,
    # loss=artifacts.LossType.CrossEntropyLoss,
)



args ('loss', 'logits', 'value.3', 'key.11', 'value.11', 'key.19', 'value.19', 'key.27', 'value.27', 'key.35', 'value.35', 'key.43', 'value.43')
args after edit ['loss', 'logits']


InferenceError: [ShapeInferenceError] (op_type:SoftmaxCrossEntropyLoss, node name: onnx::SoftmaxCrossEntropyLoss::10): labels typestr: Tind, has unsupported type: tensor(float)

In [13]:
## test generated artifacts
# create checkpoint state

state = ort_api.CheckpointState.load_checkpoint("checkpoint")

model = ort_api.Module('training_model.onnx', state, 'eval_model.onnx')

optimizer = ort_api.Optimizer('optimizer_model.onnx', model)

In [14]:
import numpy as np
import pandas as pd

In [15]:
df = pd.read_csv('../data/dialogueText.csv')

In [17]:
print(df.text)

0          Hello folks, please help me a bit with the fol...
1          Did I choose a bad channel? I ask because you ...
2          the second sentence is better english   and we...
3                                               Sock Puppe?t
4                                                       WTF?
                                 ...                        
1038319                                           anyone on?
1038320                                                  yes
1038321    can I get a pastebin of someones menu.lst with...
1038322                         http://pastebin.com/fe921690
1038323                                               thanks
Name: text, Length: 1038324, dtype: object
